In [1]:
# 1. load data
# 2. clean data according to the tuning plan (adjust data)
# 3. create features table 
# 4. test RFM price model 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#read dataset
houses = pd.read_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/houses.csv')

In [3]:
# delete globalId.1 and change columns names
del houses['globalId.1']

houses.columns = ['globalId', 'publicationDate', 'zipcode', 'sellingPrice', 'description', 'houseType', 
'categoryObject', 'constructionYear', 'garden', 'parcel', 'office', 'rooms', 'bathrooms', 
'energyLabel', 'livingArea', 'signDate']

In [4]:
# replace nan
houses['bathrooms'] = houses['bathrooms'].replace(np.NaN, houses["bathrooms"].mean())
houses['sellingPrice'] = houses['sellingPrice'].replace(np.NaN, houses["sellingPrice"].mean())

#Round # of bathrooms
houses['bathrooms']=houses['bathrooms'].round(decimals=0)

# set threshold house price 50000
houses = houses.drop(houses[houses['sellingPrice']<=50000].index)

#remove outlier oppervlakte (1)
meansurface = houses['livingArea'].mean()
houses['livingArea'].replace(1, meansurface, inplace=True)

# remove part of the output
houses['constructionYear'] = houses['constructionYear'].str[-4:]
houses['zipcode_numbers'] = [i[:4] for i in houses['zipcode']] #store in new column

# split soortWoning and encode soortwoning1
houses[['houseType1','houseType2']] = houses.houseType.str.split("}>",1,expand=True)
houses['houseType1'] = houses['houseType1'].str[2:]

# change dtype
houses['publicationDate'] = pd.to_datetime(houses['publicationDate'])
houses['signDate'] = pd.to_datetime(houses['signDate'])

In [5]:
# remove outliers from sellingPrice
houses["sellingPrice"] = np.where(houses["sellingPrice"] <139000.0, 139000.0, houses['sellingPrice'])
houses["sellingPrice"] = np.where(houses["sellingPrice"] >650000.0, 650000.0, houses['sellingPrice'])

In [6]:
# replace NaN with livingArea in parcel 
houses.parcel.fillna(houses.livingArea, inplace=True)

In [7]:
# check missing values from parcel
houses['parcel'].isnull().sum()

0

In [8]:
# dummy code categoryObject
houses = pd.get_dummies(houses, columns = ['categoryObject'])

In [9]:
# change numeric to object
houses['garden'] = houses['garden'].astype('object')
# dummy code garden
houses = pd.get_dummies(houses, columns = ['garden'])

In [10]:
# replace missing values energy label 
houses['energyLabel'] = houses['energyLabel'].fillna(0)
# dummy code energy label
houses = pd.get_dummies(houses, columns = ['energyLabel'])

In [11]:
# replace missing values houses types
houses['houseType1'] = houses['houseType1'].fillna(0)
# dummy code houseType
houses = pd.get_dummies(houses, columns = ['houseType1'])

# replace missing values houses types
houses['houseType2'] = houses['houseType2'].fillna(0)
# dummy code houseType
houses = pd.get_dummies(houses, columns = ['houseType2'])


In [12]:
#Calculate object age
houses['constructionYear'] = [i[:4] for i in houses['constructionYear']]
houses['constructionYear'] = pd.to_numeric(houses['constructionYear'])
houses['objectAge'] = 2018 - houses['constructionYear']

In [13]:
#Calculate sellingtime
houses['publicationDate'] = pd.to_datetime(houses['publicationDate'])
houses['signDate'] = pd.to_datetime(houses['signDate'])

houses['sellingtime'] = houses['signDate'] - houses['publicationDate']
houses['sellingtime'] = houses['sellingtime'].dt.days.astype('int16')

In [14]:
#Removing columns that are not needed
del houses['globalId']
del houses['publicationDate']
del houses['description']
del houses['office']
del houses['signDate']
del houses['constructionYear']
del houses['zipcode_numbers']
del houses['houseType']

In [15]:
features = houses

In [16]:
features

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,houseType2_ <{vrijstaande woning}> (<{hofjeswoning}>),houseType2_ <{vrijstaande woning}> (<{kwadrantwoning}>),houseType2_ <{vrijstaande woning}> (<{paalwoning}>),houseType2_ <{vrijstaande woning}> (<{patiowoning}>),houseType2_ <{vrijstaande woning}> (<{semi-bungalow}>),houseType2_ <{vrijstaande woning}> (<{split-level woning}>),houseType2_ <{vrijstaande woning}> (<{waterwoning}>),houseType2_ <{vrijstaande woning}> (<{wind/watermolen}>),objectAge,sellingtime
0,1774PG,139000.0,62.0,3,1.0,62.0,0,1,0,1,...,0,0,0,0,0,0,0,0,38,104
1,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,0,0,0,0,0,0,0,0,38,-25
2,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,0,0,0,0,0,0,0,0,8,113
3,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,0,0,0,0,0,0,0,0,45,132
4,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,0,0,0,0,0,0,0,0,118,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211612,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,0,0,0,0,0,0,0,0,44,66
211613,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,0,0,0,0,0,0,0,0,23,17
211614,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,0,0,0,0,1,0,0,0,70,215
211615,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,0,0,0,0,0,0,0,0,12,98


In [18]:
#################### ADDING CBS DATA ##########
##############################################
#############################################
import pandas as pd
import numpy as np

In [19]:
#The code you need starts here

#read dataset
#features = pd.read_csv('/Volumes/Untitled/MDDB/02-03 Artificial Intelligence for Business/Raw data/clean.csv', sep=';')
codes = pd.read_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/Others/codes.csv', sep=';')

In [20]:
#Checking rowcount / not needed in final code
features
#211.583 rows

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,houseType2_ <{vrijstaande woning}> (<{hofjeswoning}>),houseType2_ <{vrijstaande woning}> (<{kwadrantwoning}>),houseType2_ <{vrijstaande woning}> (<{paalwoning}>),houseType2_ <{vrijstaande woning}> (<{patiowoning}>),houseType2_ <{vrijstaande woning}> (<{semi-bungalow}>),houseType2_ <{vrijstaande woning}> (<{split-level woning}>),houseType2_ <{vrijstaande woning}> (<{waterwoning}>),houseType2_ <{vrijstaande woning}> (<{wind/watermolen}>),objectAge,sellingtime
0,1774PG,139000.0,62.0,3,1.0,62.0,0,1,0,1,...,0,0,0,0,0,0,0,0,38,104
1,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,0,0,0,0,0,0,0,0,38,-25
2,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,0,0,0,0,0,0,0,0,8,113
3,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,0,0,0,0,0,0,0,0,45,132
4,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,0,0,0,0,0,0,0,0,118,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211612,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,0,0,0,0,0,0,0,0,44,66
211613,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,0,0,0,0,0,0,0,0,23,17
211614,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,0,0,0,0,1,0,0,0,70,215
211615,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,0,0,0,0,0,0,0,0,12,98


In [23]:
codes

,PC6,Buurt2018,Wijk2018,Gemeente2018
7,1011AB,3630400,36304,363
16,1011AC,3630400,36304,363
26,1011AD,3630400,36304,363
29,1011AE,3630400,36304,363
35,1011AG,3630403,36304,363
...,...,...,...,...
7646071,9999XG,16510203,165102,1651
7646078,9999XH,16510209,165102,1651
7646084,9999XJ,16510209,165102,1651
7646094,9999XK,16510203,165102,1651


In [22]:
#######################CLEANING CODES#####################
# Deleting the collumn huisnummer
codes = codes.drop(columns="Huisnummer")

# sorting by first name
codes.sort_values("PC6", inplace = True)

# dropping ALL duplicte values
codes.drop_duplicates(subset=["PC6"], keep = 'last', inplace = True)

In [24]:
#Merge features with codes
features_codes = pd.merge(features, codes, how='left', left_on='zipcode', right_on='PC6')

In [25]:
#checking rowcount / not needed in final code
features_codes
#211.583

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,houseType2_ <{vrijstaande woning}> (<{semi-bungalow}>),houseType2_ <{vrijstaande woning}> (<{split-level woning}>),houseType2_ <{vrijstaande woning}> (<{waterwoning}>),houseType2_ <{vrijstaande woning}> (<{wind/watermolen}>),objectAge,sellingtime,PC6,Buurt2018,Wijk2018,Gemeente2018
0,1774PG,139000.0,62.0,3,1.0,62.0,0,1,0,1,...,0,0,0,0,38,104,NaN,NaN,NaN,NaN
1,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,0,0,0,0,38,-25,7481LK,1580001.0,15800.0,158.0
2,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,0,0,0,0,8,113,1068MS,3638103.0,36381.0,363.0
3,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,0,0,0,0,45,132,5628EN,7721531.0,77215.0,772.0
4,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,0,0,0,0,118,123,7731TV,1750106.0,17501.0,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211578,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,0,0,0,0,44,66,9791GD,90000.0,900.0,9.0
211579,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,0,0,0,0,23,17,6431GT,9171201.0,91712.0,917.0
211580,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,1,0,0,0,70,215,4051EW,17400100.0,174001.0,1740.0
211581,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,0,0,0,0,12,98,4043GG,17400001.0,174000.0,1740.0


In [26]:
#Remove data records that did not match with codes
#reason being we would not be able to add cbs information anyway.
features_codes = features_codes[features_codes['PC6'].notna()]

In [27]:
#Checking the rowcount / not needed in final code
features_codes
#205.794

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,houseType2_ <{vrijstaande woning}> (<{semi-bungalow}>),houseType2_ <{vrijstaande woning}> (<{split-level woning}>),houseType2_ <{vrijstaande woning}> (<{waterwoning}>),houseType2_ <{vrijstaande woning}> (<{wind/watermolen}>),objectAge,sellingtime,PC6,Buurt2018,Wijk2018,Gemeente2018
1,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,0,0,0,0,38,-25,7481LK,1580001.0,15800.0,158.0
2,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,0,0,0,0,8,113,1068MS,3638103.0,36381.0,363.0
3,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,0,0,0,0,45,132,5628EN,7721531.0,77215.0,772.0
4,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,0,0,0,0,118,123,7731TV,1750106.0,17501.0,175.0
5,5971CR,162500.0,104.0,4,1.0,68.0,0,1,0,1,...,0,0,0,0,48,243,5971CR,15070802.0,150708.0,1507.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211578,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,0,0,0,0,44,66,9791GD,90000.0,900.0,9.0
211579,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,0,0,0,0,23,17,6431GT,9171201.0,91712.0,917.0
211580,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,1,0,0,0,70,215,4051EW,17400100.0,174001.0,1740.0
211581,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,0,0,0,0,12,98,4043GG,17400001.0,174000.0,1740.0


In [28]:
### Add additional data from cbs 
services = pd.read_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/Others/services.csv', sep=';')
infrastructure = pd.read_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/Others/infrastructure.csv', sep=';')
leisure = pd.read_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/Others/leisure.csv', sep=';')

In [29]:
###################CLEANING SERVICES##################
#trim the spaces for column AfstandTotGroteSupermarkt_96
services['AfstandTotGroteSupermarkt_96'] = services['AfstandTotGroteSupermarkt_96'].str.strip()
#replace the '.' for blank values in column AfstandTotGroteSupermarkt_96
services['AfstandTotGroteSupermarkt_96'] = services['AfstandTotGroteSupermarkt_96'].replace(['.'], np.nan)
#trim the spaces for column AfstandTotSchool_98
services['AfstandTotSchool_98'] = services['AfstandTotSchool_98'].str.strip()
#replace the '.' for blank values in column AfstandTotSchool_98
services['AfstandTotSchool_98'] = services['AfstandTotSchool_98'].replace(['.'], np.nan)
#trim the spaces for column ScholenBinnen3Km_99
services['ScholenBinnen3Km_99'] = services['ScholenBinnen3Km_99'].str.strip()
#replace the '.' for blank values in column ScholenBinnen3Km_99
services['ScholenBinnen3Km_99'] = services['ScholenBinnen3Km_99'].replace(['.'], np.nan)
#delete first two characters for all rows in column Codering_3
services['Codering_3'] = [i[2:] for i in services['Codering_3']]
#delete column WijkenEnBuurten
del services['WijkenEnBuurten']
#change datatype from object to numeric for Codering_3
services['Codering_3'] = pd.to_numeric(services['Codering_3'])
#change datatype from object to numeric for AfstandTotGroteSupermarkt_96
services['AfstandTotGroteSupermarkt_96'] = pd.to_numeric(services['AfstandTotGroteSupermarkt_96'])
#change datatype from object to numeric for AfstandTotSchool_98
services['AfstandTotSchool_98'] = pd.to_numeric(services['AfstandTotSchool_98'])
#change datatype from object to numeric for ScholenBinnen3Km_99
services['ScholenBinnen3Km_99'] = pd.to_numeric(services['ScholenBinnen3Km_99'])

#trim spaces gemeentenaam
services['Gemeentenaam_1'] = services['Gemeentenaam_1'].str.strip()

#trim spaces soortregio
services['SoortRegio_2'] = services['SoortRegio_2'].str.strip()


##################CLEANING INFRASTRUCTURE#####################
infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'] = infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'].str.strip()
infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'] = infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'].replace(['.'], np.nan)

infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'] = infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'].str.strip()
infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'] = infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'].replace(['.'], np.nan)

infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'] = infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'].str.strip()
infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'] = infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'].replace(['.'], np.nan)

infrastructure['Regioaanduiding/Gemeentenaam (naam)'] = infrastructure['Regioaanduiding/Gemeentenaam (naam)'].str.strip()

infrastructure['Regioaanduiding/Soort regio (omschrijving)'] = infrastructure['Regioaanduiding/Soort regio (omschrijving)'].str.strip()

infrastructure['Regioaanduiding/Codering (code)'] = infrastructure['Regioaanduiding/Codering (code)'].str.strip()

infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'] = pd.to_numeric(infrastructure['Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)'])
infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'] = pd.to_numeric(infrastructure['Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)'])
infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'] = pd.to_numeric(infrastructure['Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)'])

infrastructure['coding'] = infrastructure['Regioaanduiding/Codering (code)']
infrastructure['coding'] = [i[2:] for i in infrastructure['coding']]
#belowd added to be able to merge float on float
infrastructure['coding'] = infrastructure.coding.astype(float)

del infrastructure['Wijken en buurten']

#################CLEANING LEISURE##################
leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'] = leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'].str.strip()
leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'] = leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'].replace(['.'], np.nan)

leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'] = leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'].str.strip()
leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'] = leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'].replace(['.'], np.nan)

leisure['Horeca/Restaurants/Afstand tot restaurant (km)'] = leisure['Horeca/Restaurants/Afstand tot restaurant (km)'].str.strip()
leisure['Horeca/Restaurants/Afstand tot restaurant (km)'] = leisure['Horeca/Restaurants/Afstand tot restaurant (km)'].replace(['.'], np.nan)

leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'] = leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'].str.strip()
leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'] = leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'].replace(['.'], np.nan)

leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'] = leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'].str.strip()
leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'] = leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'].replace(['.'], np.nan)

leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'] = leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'].str.strip()
leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'] = leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'].replace(['.'], np.nan)

leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'] = leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'].str.strip()
leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'] = leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'].replace(['.'], np.nan)

leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'] = leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'].str.strip()
leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'] = leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'].replace(['.'], np.nan)

leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'] = leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'].str.strip()
leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'] = leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'].replace(['.'], np.nan)

leisure['Regioaanduiding/Gemeentenaam (naam)'] = leisure['Regioaanduiding/Gemeentenaam (naam)'].str.strip()

leisure['Regioaanduiding/Soort regio (omschrijving)'] = leisure['Regioaanduiding/Soort regio (omschrijving)'].str.strip()

leisure['Regioaanduiding/Codering (code)'] = leisure['Regioaanduiding/Codering (code)'].str.strip()

del leisure['Wijken en buurten']

leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'] = pd.to_numeric(leisure['Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)'])

leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'] = pd.to_numeric(leisure['Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)'])

leisure['Horeca/Restaurants/Afstand tot restaurant (km)'] = pd.to_numeric(leisure['Horeca/Restaurants/Afstand tot restaurant (km)'])

leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'] = pd.to_numeric(leisure['Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)'])

leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'] = pd.to_numeric(leisure['Vrije tijd en cultuur/Afstand tot bibliotheek (km)'])

leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'] = pd.to_numeric(leisure['Vrije tijd en cultuur/Museum/Afstand tot museum (km)'])

leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'] = pd.to_numeric(leisure['Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)'])

leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'] = pd.to_numeric(leisure['Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)'])

leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'] = pd.to_numeric(leisure['Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)'])

leisure['coding'] = leisure['Regioaanduiding/Codering (code)']
leisure['coding'] = [i[2:] for i in leisure['coding']]
leisure['coding'] = pd.to_numeric(leisure['coding'])

In [30]:
#### merge feature_codes with services 
features_codes_services = pd.merge(features_codes, services, how='left', left_on='Buurt2018', right_on='Codering_3')

In [31]:
#### merge feature_codes_services with infrastructure
features_codes_services_infrastructure = pd.merge(features_codes_services, infrastructure, how='left', left_on='Codering_3', right_on='coding')

In [32]:
#### merge features_codes_services_infrastructure with leisure
features_cbs = pd.merge(features_codes_services_infrastructure, leisure, how='left', left_on='Codering_3', right_on='coding')

In [33]:
#checking rowcount / not needed in final code
features_cbs
#207.754

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,Horeca/Cafés en dergelijke/Afstand tot café e.d. (km),Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal),Horeca/Restaurants/Afstand tot restaurant (km),Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal),Vrije tijd en cultuur/Afstand tot bibliotheek (km),Vrije tijd en cultuur/Museum/Afstand tot museum (km),Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal),Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km),Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal),coding_y
0,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,0.3,7.0,0.4,19.6,0.7,0.8,14.0,0.7,4.0,1580001
1,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,0.5,15.0,0.4,68.0,0.8,2.3,61.0,5.4,18.0,3638103
2,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,1.8,3.8,0.6,20.3,5.0,5.1,17.1,5.0,8.6,7721531
3,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,5.0,0.1,3.8,0.3,5.9,6.4,7.4,13.2,1.7,1750106
4,5971CR,162500.0,104.0,4,1.0,68.0,0,1,0,1,...,0.5,4.0,0.4,5.0,2.4,7.7,7.8,7.5,5.0,15070802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207749,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,0.7,1.1,0.6,5.0,0.6,11.7,10.9,12.0,3.0,90000
207750,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,0.4,25.6,0.5,23.1,1.1,2.3,14.0,6.0,5.0,9171201
207751,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,0.8,1.1,0.9,4.1,0.6,6.9,11.0,11.6,4.0,17400100
207752,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,0.7,2.0,1.8,3.1,0.6,7.8,13.0,8.6,5.8,17400001


In [34]:
#Deleting after joining 3 CBS tables
del features_cbs['PC6']
del features_cbs['Buurt2018']
del features_cbs['Wijk2018']
del features_cbs['Gemeente2018']
del features_cbs['ID']
del features_cbs['Gemeentenaam_1']
del features_cbs['SoortRegio_2']
del features_cbs['Codering_3']
del features_cbs['Regioaanduiding/Gemeentenaam (naam)_x']
del features_cbs['Regioaanduiding/Soort regio (omschrijving)_x']
del features_cbs['Regioaanduiding/Codering (code)_x']
del features_cbs['coding_x']
del features_cbs['Regioaanduiding/Gemeentenaam (naam)_y']
del features_cbs['Regioaanduiding/Soort regio (omschrijving)_y']
del features_cbs['Regioaanduiding/Codering (code)_y']
del features_cbs['coding_y']

In [35]:
features_cbs
#The code you need stops here
#rowcount/ not needed in final code
#207.754

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km),Horeca/Cafés en dergelijke/Afstand tot café e.d. (km),Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal),Horeca/Restaurants/Afstand tot restaurant (km),Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal),Vrije tijd en cultuur/Afstand tot bibliotheek (km),Vrije tijd en cultuur/Museum/Afstand tot museum (km),Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal),Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km),Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)
0,7481LK,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,13.3,0.3,7.0,0.4,19.6,0.7,0.8,14.0,0.7,4.0
1,1068MS,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,6.0,0.5,15.0,0.4,68.0,0.8,2.3,61.0,5.4,18.0
2,5628EN,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,4.9,1.8,3.8,0.6,20.3,5.0,5.1,17.1,5.0,8.6
3,7731TV,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,23.5,5.0,0.1,3.8,0.3,5.9,6.4,7.4,13.2,1.7
4,5971CR,162500.0,104.0,4,1.0,68.0,0,1,0,1,...,7.8,0.5,4.0,0.4,5.0,2.4,7.7,7.8,7.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207749,9791GD,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,12.5,0.7,1.1,0.6,5.0,0.6,11.7,10.9,12.0,3.0
207750,6431GT,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,6.0,0.4,25.6,0.5,23.1,1.1,2.3,14.0,6.0,5.0
207751,4051EW,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,19.8,0.8,1.1,0.9,4.1,0.6,6.9,11.0,11.6,4.0
207752,4043GG,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,15.1,0.7,2.0,1.8,3.1,0.6,7.8,13.0,8.6,5.8


In [36]:
features_cbs.isnull().sum().sum()

6630

In [44]:
# Show columns which have missing values
features_cbs.columns[features_cbs.isna().any()].tolist()

['AfstandTotHuisartsenpraktijk_95',
 'AfstandTotGroteSupermarkt_96',
 'AfstandTotKinderdagverblijf_97',
 'AfstandTotSchool_98',
 'ScholenBinnen3Km_99',
 'Verkeer en vervoer/Afstand tot oprit hoofdverkeersweg (km)',
 'Verkeer en vervoer/Treinstations/Afstand tot treinstations totaal (km)',
 'Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km)',
 'Horeca/Cafés en dergelijke/Afstand tot café e.d. (km)',
 'Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal)',
 'Horeca/Restaurants/Afstand tot restaurant (km)',
 'Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal)',
 'Vrije tijd en cultuur/Afstand tot bibliotheek (km)',
 'Vrije tijd en cultuur/Museum/Afstand tot museum (km)',
 'Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)',
 'Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)',
 'Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)']

In [45]:
# Drop all columns which have a missing value 
features_cbs_test1 = features_cbs.dropna(how = 'all')


In [49]:
features_cbs_test1

,zipcode,sellingPrice,parcel,rooms,bathrooms,livingArea,categoryObject_<{Appartement}>,categoryObject_<{Woonhuis}>,garden_0,garden_1,...,Verkeer en vervoer/Treinstations/Afstand tot belangrijk overstapstation (km),Horeca/Cafés en dergelijke/Afstand tot café e.d. (km),Horeca/Cafés en dergelijke/Aantal cafés/Binnen 3 km (aantal),Horeca/Restaurants/Afstand tot restaurant (km),Horeca/Restaurants/Aantal restaurants/Binnen 3 km (aantal),Vrije tijd en cultuur/Afstand tot bibliotheek (km),Vrije tijd en cultuur/Museum/Afstand tot museum (km),Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal),Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km),Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)
0,7481,209000.0,148.0,5,1.0,136.0,0,1,0,1,...,13.3,0.3,7.0,0.4,19.6,0.7,0.8,14.0,0.7,4.0
1,1068,267500.0,70.0,3,1.0,70.0,1,0,1,0,...,6.0,0.5,15.0,0.4,68.0,0.8,2.3,61.0,5.4,18.0
2,5628,349000.0,244.0,5,1.0,144.0,0,1,0,1,...,4.9,1.8,3.8,0.6,20.3,5.0,5.1,17.1,5.0,8.6
3,7731,495000.0,4500.0,8,1.0,323.0,0,1,1,0,...,23.5,5.0,0.1,3.8,0.3,5.9,6.4,7.4,13.2,1.7
4,5971,162500.0,104.0,4,1.0,68.0,0,1,0,1,...,7.8,0.5,4.0,0.4,5.0,2.4,7.7,7.8,7.5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207749,9791,159000.0,195.0,5,1.0,116.0,0,1,0,1,...,12.5,0.7,1.1,0.6,5.0,0.6,11.7,10.9,12.0,3.0
207750,6431,179000.0,160.0,4,1.0,84.0,0,1,0,1,...,6.0,0.4,25.6,0.5,23.1,1.1,2.3,14.0,6.0,5.0
207751,4051,391500.0,465.0,5,1.0,99.0,0,1,0,1,...,19.8,0.8,1.1,0.9,4.1,0.6,6.9,11.0,11.6,4.0
207752,4043,375000.0,300.0,7,1.0,172.0,0,1,0,1,...,15.1,0.7,2.0,1.8,3.1,0.6,7.8,13.0,8.6,5.8


In [51]:
features_cbs_test1['zipcode'] = [i[:4] for i in features_cbs_test1['zipcode']] #store in new column
features_cbs_test1['zipcode'] = pd.to_numeric(features_cbs_test1['zipcode'])

In [53]:
features_cbs_test1.dtypes

zipcode                                                                    int64
sellingPrice                                                             float64
parcel                                                                   float64
rooms                                                                      int64
bathrooms                                                                float64
                                                                          ...   
Vrije tijd en cultuur/Afstand tot bibliotheek (km)                       float64
Vrije tijd en cultuur/Museum/Afstand tot museum (km)                     float64
Vrije tijd en cultuur/Museum/Aantal musea/Binnen 20 km (aantal)          float64
Vrije tijd en cultuur/Bioscoop/Afstand tot bioscoop (km)                 float64
Vrije tijd en cultuur/Bioscoop/Aantal bioscopen/Binnen 20 km (aantal)    float64
Length: 168, dtype: object

In [37]:
##features_cbs.to_csv('/Users/christianberbig/Documents/Digital Driven Business /Github/Raw data/features_cbs.csv', sep=';', index=False)

In [ ]:
### The only thing that is left to do here is to potentially impute some of the cbs data with the mean (option 1) and compare that with a version where you drop the records that do not have all complete CBS data (option 2)

In [39]:
################# RANDOM FOREST MODEL #####################
##########################################################
#########################################################

# Libraries and options
# If error [No module named 'sklearn'], in terminal: conda install -c conda-forge scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
%matplotlib notebook
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
#from xgboost import XGBRegressor

In [54]:
x = features_cbs_test1.loc[:, features.columns != 'sellingPrice']
y = features_cbs_test1['sellingPrice']

IndexError: Boolean index has wrong length: 151 instead of 168

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=24)

In [ ]:
x.shape

In [ ]:
model = RandomForestRegressor(min_samples_split=5, n_estimators=20)
model.fit(x_train,y_train)

In [ ]:
# Test the model
RF_prediction = model.predict(x_test)

# Compute errors
MAE = mean_absolute_error(y_test , RF_prediction)
print('Random Forest MAE = ', MAE)